In [ ]:
!pip install mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 11.6 MB/s eta 0:00:00


In [2]:
import cv2
import mediapipe as mp
import time
import numpy as np
from google.colab.patches import cv2_imshow

In [21]:
# Initialize MediaPipe components globally for holistic model and drawing utilities
mp_holistic = mp.solutions.holistic  # MediaPipe Holistic for full-body landmark detection
mp_drawing = mp.solutions.drawing_utils  # Utility to draw landmarks and connections

In [22]:
def initialize_video(video_path):
    """
    Initialize video capture, output settings, and check video properties.
    Args:
        video_path (str): Path to the input video file.
    Returns:
        cap: VideoCapture object for reading video frames.
        out: VideoWriter object to save processed video.
        frame_width (int): Width of the video frames.
        frame_height (int): Height of the video frames.
        fps (int): Frames per second of the input video.
    """
    cap = cv2.VideoCapture(video_path)  # Capture video from the file
    if not cap.isOpened():  # Check if video file is loaded successfully
        print("Error: Could not open video file.")
        exit()

    # Extract video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Set up output video file properties
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4 format
    out = cv2.VideoWriter('Body_Landmarks.mp4', fourcc, fps, (frame_width, frame_height))

    return cap, out, frame_width, frame_height, fps

In [23]:
def define_dense_connections():
    """
    Define the dense connections between body landmarks for enhanced visualization.
    Returns:
        list of tuples: Dense landmark connections for drawing lines between key points.
    """
    return [
        # Dense body connections between shoulders, elbows, wrists, hips, knees, and ankles
        (mp_holistic.PoseLandmark.LEFT_SHOULDER.value, mp_holistic.PoseLandmark.RIGHT_SHOULDER.value),
        (mp_holistic.PoseLandmark.LEFT_SHOULDER.value, mp_holistic.PoseLandmark.LEFT_ELBOW.value),
        (mp_holistic.PoseLandmark.LEFT_ELBOW.value, mp_holistic.PoseLandmark.LEFT_WRIST.value),
        (mp_holistic.PoseLandmark.RIGHT_SHOULDER.value, mp_holistic.PoseLandmark.RIGHT_ELBOW.value),
        (mp_holistic.PoseLandmark.RIGHT_ELBOW.value, mp_holistic.PoseLandmark.RIGHT_WRIST.value),
        (mp_holistic.PoseLandmark.LEFT_HIP.value, mp_holistic.PoseLandmark.RIGHT_HIP.value),
        (mp_holistic.PoseLandmark.LEFT_HIP.value, mp_holistic.PoseLandmark.LEFT_KNEE.value),
        (mp_holistic.PoseLandmark.LEFT_KNEE.value, mp_holistic.PoseLandmark.LEFT_ANKLE.value),
        (mp_holistic.PoseLandmark.RIGHT_HIP.value, mp_holistic.PoseLandmark.RIGHT_KNEE.value),
        (mp_holistic.PoseLandmark.RIGHT_KNEE.value, mp_holistic.PoseLandmark.RIGHT_ANKLE.value),
        (mp_holistic.PoseLandmark.LEFT_SHOULDER.value, mp_holistic.PoseLandmark.LEFT_HIP.value),
        (mp_holistic.PoseLandmark.RIGHT_SHOULDER.value, mp_holistic.PoseLandmark.RIGHT_HIP.value),
    ]

In [24]:
def process_frame(frame, holistic, dense_body_connections, frame_width, frame_height):
    """
    Process a single video frame to detect landmarks and draw visualizations.
    Args:
        frame: The input frame.
        holistic: MediaPipe Holistic model object.
        dense_body_connections (list): List of landmark connections to draw.
        frame_width (int): Width of the frame.
        frame_height (int): Height of the frame.
    Returns:
        black_frame: A black frame with the detected landmarks and connections drawn.
    """
    # Create a black canvas of the same size as the input frame
    black_frame = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)

    # Convert the BGR image to RGB (required for MediaPipe processing)
    imgRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = holistic.process(imgRGB)  # Process the frame to detect landmarks

    # Draw body landmarks
    if results.pose_landmarks:
        draw_dense_body_connections(black_frame, results, dense_body_connections, frame_width, frame_height)
        draw_body_landmark_points(black_frame, results, frame_width, frame_height)

    # Draw hands and face landmarks
    draw_hands_and_face(black_frame, results)

    return black_frame

In [25]:
def draw_dense_body_connections(black_frame, results, dense_body_connections, frame_width, frame_height):
    """
    Draw dense lines connecting body landmarks.
    """
    for connection in dense_body_connections:
        start_idx, end_idx = connection
        start = results.pose_landmarks.landmark[start_idx]
        end = results.pose_landmarks.landmark[end_idx]
        start_point = (int(start.x * frame_width), int(start.y * frame_height))
        end_point = (int(end.x * frame_width), int(end.y * frame_height))
        cv2.line(black_frame, start_point, end_point, (255, 255, 0), thickness=2)

In [26]:
def draw_body_landmark_points(black_frame, results, frame_width, frame_height):
    """
    Draw small circles on each body landmark.
    """
    for landmark in results.pose_landmarks.landmark:
        x = int(landmark.x * frame_width)
        y = int(landmark.y * frame_height)
        cv2.circle(black_frame, (x, y), radius=3, color=(0, 255, 255), thickness=-1)

In [27]:
def draw_hands_and_face(black_frame, results):
    """
    Draw hands and face landmarks using MediaPipe drawing utilities.
    """
    if results.left_hand_landmarks:
        mp_drawing.draw_landmarks(
            black_frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2)
        )
    if results.right_hand_landmarks:
        mp_drawing.draw_landmarks(
            black_frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(255, 0, 255), thickness=2, circle_radius=2)
        )
    if results.face_landmarks:
        mp_drawing.draw_landmarks(
            black_frame, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
            mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=1, circle_radius=1)
        )

In [28]:
def main(video_path):
    """
    Main function to read video, process frames, and save the output video.
    Args:
        video_path (str): Path to the input video file.
    """
    # Initialize video capture, output, and MediaPipe holistic model
    cap, out, frame_width, frame_height, fps = initialize_video(video_path)
    holistic = mp_holistic.Holistic()
    dense_body_connections = define_dense_connections()
    pTime = 0  # Initialize previous time for FPS calculation

    while True:
        success, frame = cap.read()
        if not success:  # End of video
            print("End of video file.")
            break

        # Process the frame to detect landmarks
        black_frame = process_frame(frame, holistic, dense_body_connections, frame_width, frame_height)

        # Calculate and display FPS
        cTime = time.time()
        fps_text = 1 / (cTime - pTime)
        pTime = cTime
        cv2.putText(black_frame, f"FPS= {int(fps_text)}", (10, 20), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1)

        # Write the processed frame to the output video file
        # out.write(black_frame)

        # Display the frame in Colab (optional)
        # cv2_imshow(black_frame)

    # Release resources
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print("Video processing complete. Output saved to 'Body_Landmarks.mp4'")

# Run the main function with the input video path
video_path = "/content/Raw_Video.mp4"
main(video_path)

End of video file.
Video processing complete. Output saved to 'Body_Landmarks.mp4'


In [29]:
# Run the main function
video_path = "/content/Raw_Video.mp4"
main(video_path)

End of video file.
Video processing complete. Output saved to 'Body_Landmarks.mp4'


In [13]:
from google.colab import files
files.download('/content/Body_Landmarks.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>